In [ ]:
# Import data - DELETE old file first from Files tab!
from google.colab import files
import pandas as pd
import numpy as np

uploaded=files.upload()
data = pd.read_csv('data.csv').drop(columns=['First name', 'Last name', 'Address', 'Phone', 'Email'])
data

Saving captains.csv to captains.csv
Saving data.csv to data.csv
Saving populations.csv to populations.csv


,City,County,Volunteer,Unnamed: 8
0,Bothell,King,NaN,NaN
1,Seattle,King,NaN,NaN
2,Seattle,King,NaN,NaN
3,98062,King,NaN,NaN
4,98105,King,NaN,NaN
...,...,...,...,...
6477,Olympia,Thurston,NaN,NaN
6478,Olympia,Thurston,NaN,NaN
6479,Olympia,Thurston,NaN,NaN
6480,Tumwater,Thurston,NaN,NaN


In [ ]:
# Get the signature counts per county
county_counts = pd.DataFrame(data.value_counts('County').reset_index().values, columns=['Area_name', 'Signatures'])
county_counts.Area_name += ' County'

# Match counties to geojson
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
county_data = f"{url}/us_county_data.csv"
county_geo = f"{url}/us_counties_20m_topo.json"
df = pd.read_csv(county_data, na_values=[" "]).drop(columns=['Unemployment_rate_2011', 'Employed_2011','Median_Household_Income_2011','Med_HH_Income_Percent_of_StateTotal_2011', 'Unemployed_2011', 'Civilian_labor_force_2011'])
df=df[(df.State == 'WA')]

df = pd.merge(df, county_counts, on="Area_name", how="outer").fillna(0) # how="outer" to check for data errors
print(sum(df.Signatures),'Signatures under analysis')
pd.set_option("display.max_rows", None, "display.max_columns", None)
df

6401 Signatures under analysis


,FIPS_Code,State,Area_name,Signatures
0,53000.0,WA,Washington,0
1,53001.0,WA,Adams County,0
2,53003.0,WA,Asotin County,0
3,53005.0,WA,Benton County,10
4,53007.0,WA,Chelan County,8
5,53009.0,WA,Clallam County,152
6,53011.0,WA,Clark County,318
7,53013.0,WA,Columbia County,0
8,53015.0,WA,Cowlitz County,126
9,53017.0,WA,Douglas County,2


In [ ]:
# Map absolute signature counts per county
import folium
import branca
import json
import requests

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, county_counts.Signatures.mean())
signature_series = df.set_index("FIPS_Code")["Signatures"]


def style_function(feature):
    signatures = signature_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.5,
        "weight": 0,
        "fillColor": "#black" if signatures is None else colorscale(signatures),
    }

m = folium.Map(
    location=[47.6, -120],
    zoom_start=7,
    tiles="cartodbpositron"
    )

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    "objects.us_counties_20m",
    style_function=style_function,
).add_to(m)


m

In [ ]:
# Map signatures per capita
# Get and arrange population info (download from https://worldpopulationreview.com/us-counties/states/wa)
populations = pd.read_csv('populations.csv').drop(columns=['GrowthRate', 'popDensity'])
populations.columns=['Area_name', 'Population']
df = pd.merge(df, populations, on="Area_name")

df['Signatures per Capita'] = df.apply(lambda row: row.Signatures/row.Population, axis=1)

#Map!
colorscale = branca.colormap.linear.YlOrRd_09.scale(0, df['Signatures per Capita'].mean())
signature_series = df.set_index("FIPS_Code")["Signatures per Capita"]


def style_function(feature):
    signatures = signature_series.get(int(feature["id"][-5:]), None)
    return {
        "fillOpacity": 0.5,
        "weight": 0,
        "fillColor": "#black" if signatures is None else colorscale(signatures),
    }

m = folium.Map(
    location=[47.6, -120],
    zoom_start=7,
    tiles="cartodbpositron"
    )

folium.TopoJson(
    json.loads(requests.get(county_geo).text),
    "objects.us_counties_20m",
    style_function=style_function,
).add_to(m)


m

In [ ]:
# How many signatures needed per county?
wa_population = sum(populations.Population)
signatures_needed = 400000
captains_needed = 800
populations['Signature_goal'] = np.ceil(populations.Population*(signatures_needed/wa_population))
populations['Captains_goal'] = np.ceil(populations.Population*(captains_needed/wa_population))
populations

,Area_name,Population,Signature_goal,Captains_goal
0,King County,2301620,118079.0,237.0
1,Pierce County,927428,47580.0,96.0
2,Snohomish County,840131,43101.0,87.0
3,Spokane County,541188,27765.0,56.0
4,Clark County,501869,25747.0,52.0
5,Thurston County,299496,15365.0,31.0
6,Kitsap County,275867,14153.0,29.0
7,Yakima County,251495,12903.0,26.0
8,Whatcom County,237347,12177.0,25.0
9,Benton County,210598,10805.0,22.0


In [ ]:
# Captains per county
captains_data = pd.read_csv('captains.csv')[['County']]
captains = pd.DataFrame(captains_data.value_counts('County').reset_index().values, columns=['Area_name', 'Captains'])
captains.Area_name += ' County'
progress = pd.merge(populations, captains, on='Area_name', how='outer')
progress.fillna(0, inplace=True)
progress = pd.merge(progress, df[['Area_name', 'Signatures', 'FIPS_Code']], on="Area_name")
progress['Captains_percent'] = 100*progress.Captains/progress.Captains_goal
progress['Signatures_percent'] = 100*progress.Signatures/progress.Signature_goal
progress['Signatures_per_captain_weighted'] = 100*(progress.Signatures/progress.Signature_goal)/progress.Captains
progress

,Area_name,Population,Signature_goal,Captains_goal,Captains,Signatures,FIPS_Code,Captains_percent,Signatures_percent,Signatures_per_captain_weighted
0,King County,2301620,118079.0,237.0,16,1640,53033,6.751055,1.388901,0.086806
1,Pierce County,927428,47580.0,96.0,3,279,53053,3.125000,0.586381,0.195460
2,Snohomish County,840131,43101.0,87.0,1,467,53061,1.149425,1.083502,1.083502
3,Spokane County,541188,27765.0,56.0,1,261,53063,1.785714,0.940032,0.940032
4,Clark County,501869,25747.0,52.0,3,102,53011,5.769231,0.396163,0.132054
5,Thurston County,299496,15365.0,31.0,1,60,53067,3.225806,0.390498,0.390498
6,Kitsap County,275867,14153.0,29.0,2,674,53035,6.896552,4.762241,2.381121
7,Yakima County,251495,12903.0,26.0,0,10,53077,0.000000,0.077501,inf
8,Whatcom County,237347,12177.0,25.0,2,146,53073,8.000000,1.198982,0.599491
9,Benton County,210598,10805.0,22.0,1,10,53005,4.545455,0.092550,0.092550


In [ ]:
# Island county zip codes

progress[progress['Area_name']=="Island County"]

,Area_name,Population,Signature_goal,Captains_goal,Captains,Signatures,FIPS_Code,Captains_percent,Signatures_percent,Signatures_per_captain_weighted
14,Island County,86969,4462.0,9.0,1,104,53029,11.111111,2.330793,2.330793
